# 📚 Projet – Books to Scrape

## ⏱️ Temps Estimé : **300 minutes (5 heures)**

Ce projet vous guidera dans la création d’un **web scraper** pour le site [Books to Scrape](https://books.toscrape.com/). Il est conçu comme un exercice pratique pour s’entraîner avec **Python, requests, Pandas et l’analyse de données**.

## 🎯 Contexte

L’équipe marketing d’une librairie en ligne souhaite mieux comprendre son catalogue. Elle veut collecter des informations sur tous les livres, analyser les catégories, les prix, les notes et la disponibilité en stock.

En tant que data scientist, votre mission est de **scraper le site web** et de livrer des jeux de données structurés ainsi que des premiers insights.

## ✅ Objectifs

1. Scraper le site [Books to Scrape](https://books.toscrape.com/)
2. Extraire pour chaque livre :
   - Titre
   - Prix
   - Disponibilité en stock
   - Note
   - URL du produit
   - URL de l’image
   - UPC
   - Catégorie
3. Gérer la **pagination** sur toutes les pages
4. Sauvegarder les résultats dans **un CSV par catégorie**
5. Télécharger les images des couvertures de livres dans des dossiers par catégorie


## 📦 Livrables

- Fichiers CSV : `outputs/csv/category_<slug>.csv`
- Images : `outputs/images/<category>/<upc>_<slug-title>.jpg`
- Optionnel : Un notebook de nettotage et d’exploration qui analyse les prix, notes et stocks avec Pandas et Quelques visualisation à réaliser avec les packages que vous préféré.

## 🛠 Étapes suggérées

1. Commencez par scraper **une page produit** et extraire les champs demandés
2. Étendez votre code à **une catégorie** (gestion de plusieurs pages)
3. Généralisez votre scraper à **toutes les catégories**
4. Sauvegardez les résultats dans des fichiers CSV
5. Étendez le scraper pour aussi **télécharger les images**
6. (Optionnel) Explorez le dataset avec Pandas (prix moyen par catégorie, distribution des notes, etc.)

# 🛠 Étapes détaillées

## 🟢 Phase 1 – Construire pas à pas (dans un seul script au début)
1. **Récupérer une page** → utiliser `requests` pour télécharger le HTML.  
2. **Extraire les titres** → avec `Selector`, récupérer les noms des livres sur la page d’accueil.  
3. **Extraire les détails** → ouvrir la page d’un livre et extraire :
   - titre  
   - prix  
   - disponibilité en stock  
   - note (rating)  
   - UPC  
   - **URL de l’image** (utiliser `.css("img::attr(src)")` + `urljoin` pour obtenir le lien absolu)  
4. **Gérer une catégorie** → collecter toutes les URLs de livres dans une page de catégorie.  
5. **Gérer plusieurs pages** → suivre le bouton `"li.next a"` jusqu’à ce qu’il n’y en ait plus.  
6. **Sauvegarder les résultats** → écrire les résultats dans un fichier CSV (`outputs/csv/category_<name>.csv`).  
7. **Télécharger les images** → utiliser l’URL de l’image pour la télécharger avec `requests` :
   - Chemin : `outputs/images/<categorie>/<upc>_<slug-title>.jpg`  
8. **Gérer plusieurs catégories** → boucler sur toutes les catégories depuis la page d’accueil.  


## 🟡 Phase 2 – Organiser le projet
Une fois que le code fonctionne, séparer en plusieurs fichiers :  
- `parsers.py` → fonctions de scraping (par ex. `parse_list_page`, `parse_product_page`, `get_category_links`)  
- `utils.py` → fonctions utilitaires (par ex. `write_csv`, `download_file`, `ensure_dir`)  
- `settings.py` → constantes (`BASE_URL`, `HEADERS`, `DEFAULT_DELAY`, `TIMEOUT`)  
- `scrape.py` → script principal (seulement `main()` + argparse), appelle les fonctions des autres fichiers  


## 🔵 Phase 3 – Automatiser avec la ligne de commande (CLI)
Ajouter des **options argparse** dans `scrape.py` :  
- `--categories Travel,Poetry` → scraper seulement certaines catégories  
- `--max-pages 1` → limiter le scraping pour des tests rapides  
- `--delay 1` → ajouter un délai entre les requêtes  
- `--outdir outputs` → changer le dossier de sortie  

Exemple d’utilisation :
```bash
python scrape.py --categories Travel --max-pages 1


In [69]:
import requests
from parsel import Selector
import pandas as pd
import os
import time

# URL de base
url = "https://books.toscrape.com/"


In [70]:
# Faire la requête
r = requests.get(url)
print("Status code:", r.status_code)

# Créer le sélecteur
response = Selector(text=r.text)


Status code: 200


In [71]:
# Tester sur un livre
book = response.css('article.product_pod')[0]

# Titre
title = book.css('h3 a::attr(title)').get()
print("Titre:", title)

# Prix
price = book.css('p.price_color::text').get()
print("Prix:", price)

# Note (rating)
rating_class = book.css('p.star-rating::attr(class)').get()
print("Rating class:", rating_class)

# Disponibilité (nettoyée)
availability = book.css('p.instock.availability::text').getall()
availability_clean = [a.strip() for a in availability if a.strip()]
print("Disponibilité:", availability_clean)

# URL du livre
book_url = book.css('h3 a::attr(href)').get()
print("URL:", book_url)

# URL de l'image
image_url = book.css('div.image_container img::attr(src)').get()
print("Image:", image_url)


Titre: A Light in the Attic
Prix: Â£51.77
Rating class: star-rating Three
Disponibilité: ['In stock']
URL: catalogue/a-light-in-the-attic_1000/index.html
Image: media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg


Rating: 3 étoiles


Nombre de livres sur la page: 20
Total livres extraits: 20

Dix premier livres:
[{'title': 'A Light in the Attic', 'price': 'Â£51.77', 'rating': 3, 'in_stock': True, 'url': 'catalogue/a-light-in-the-attic_1000/index.html', 'image': 'media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg'}, {'title': 'Tipping the Velvet', 'price': 'Â£53.74', 'rating': 1, 'in_stock': True, 'url': 'catalogue/tipping-the-velvet_999/index.html', 'image': 'media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg'}, {'title': 'Soumission', 'price': 'Â£50.10', 'rating': 1, 'in_stock': True, 'url': 'catalogue/soumission_998/index.html', 'image': 'media/cache/3e/ef/3eef99c9d9adef34639f510662022830.jpg'}, {'title': 'Sharp Objects', 'price': 'Â£47.82', 'rating': 4, 'in_stock': True, 'url': 'catalogue/sharp-objects_997/index.html', 'image': 'media/cache/32/51/3251cf3a3412f53f339e42cac2134093.jpg'}, {'title': 'Sapiens: A Brief History of Humankind', 'price': 'Â£54.23', 'rating': 5, 'in_stock': True, 'url': 'catalogue/

## 🟣 Phase 4 – Documenter
- Créer un fichier `README.md` avec :
  - L’objectif du projet et son contexte  
  - Comment installer les dépendances (`pip install -r requirements.txt`)  
  - Comment exécuter le scraper  
  - Quelques exemples de commandes  


## 🔴 Phase 5 – Partager
- Publier votre projet sur GitHub avec :
  - Les fichiers de code (`scrape.py`, `parsers.py`, `utils.py`, `settings.py`)  
  - Un exemple de `README.md`  
  - Un dossier `outputs/` vide avec un fichier `.gitkeep` pour conserver la structure  


## 🟠 Phase 6 – Explorer les données
Ouvrir un **notebook Jupyter** pour analyser les données extraites :

1. **Charger un CSV** avec Pandas :
   ```python
   import pandas as pd
   df = pd.read_csv("outputs/csv/category_travel.csv")
   df.head()


## 📝 Critères d’évaluation

- 💡 Effort et compréhension :  
  - Je valorise **vos propres essais** plus qu’un simple copier-coller depuis ChatGPT ou Internet.  
  - Même un progrès partiel, des commentaires clairs dans le code, ou différentes tentatives montrent un véritable apprentissage.  
  - Vous devez être capable d’**expliquer votre code** lors de la relecture ou de la présentation.  

- 🌟 Points bonus pour la partie analyse de la donnée à la fin avec un notebook et de la recherche sur itnernet de comment faire des visualisations pertinentes et créative dans le notebook (vous pouvez explorer la librarie **plotly express**).  
